In [48]:
import pandas as pd

import pyteomics.mgf
from tqdm.notebook import tqdm

In [49]:
infile = r"C:\git\msn_library\data\gnpslib\ALL_GNPS_NO_PROPOGATED.mgf"
df_file = r"C:\git\msn_library\data\final_tables\gnps_phase.tsv"

### Group all rows with the same content
This needs to move to the script that generates the tsv file in the first place

In [ ]:
filtered_df = pd.read_csv(df_file, sep='\t')
filtered_df = filtered_df.rename(columns={"clinical_phase": "clinical_phase_no_drugbank", "clinical_phase2": "clinical_phase"})
filtered_df = filtered_df[filtered_df["clinical_phase"]>0]
filtered_df["none"] = filtered_df.isnull().sum(axis=1)
filtered_df = filtered_df.sort_values(by="none", ascending=True).drop_duplicates(['gnps_libid']).sort_values(by="clinical_phase", ascending=False)

cols = [col for col in filtered_df.columns if col not in ["gnps_libid", "entries"]]
filtered_df = filtered_df.groupby(cols, dropna=False)['gnps_libid'].apply(', '.join).reset_index()
filtered_df["entries"] = [str(lib).count(",") + 1 for lib in filtered_df["gnps_libid"]]
filtered_df.sort_values(["clinical_phase", "inchi_key"], ascending=[False, True], inplace=True)

filtered_df.to_csv(r"C:\git\msn_library\data\final_tables\gnps_drugs_clinical_phase_greater0.tsv", sep="\t", index=False)

### Start creating mgf from library

In [50]:
# read tsv
filtered_df = pd.read_csv(df_file, sep='\t')
filtered_df = filtered_df.rename(columns={"clinical_phase": "clinical_phase_no_drugbank", "clinical_phase2": "clinical_phase"})
filtered_df = filtered_df[filtered_df["clinical_phase"]>0]
filtered_df["none"] = filtered_df.isnull().sum(axis=1)
filtered_df = filtered_df.sort_values(by="none", ascending=True).drop_duplicates(['gnps_libid']).sort_values(by="clinical_phase", ascending=False)
filtered_df['gnps_libid'] = [ids.split(", ") for ids in filtered_df['gnps_libid']]
filtered_df = filtered_df.explode(column='gnps_libid')
filtered_df.set_index('gnps_libid', inplace=True)
# filtered_df[filtered_df.duplicated(['gnps_libid'], keep=False)]
filtered_df

,Smiles,compound_name,entries,synonyms,pubchem_cid,isomeric_smiles,canonical_smiles,pubchem_cid_parent,iupac,pubchem_logp,...,drugcentral_status,drugcentral_database_id,drugcentral_date_of_approval,drugcentral_administration,drugcentral_orphan,drugbank_approved_number,drugcentral_administration_number,clinical_phase,any_phase,none
gnps_libid,,,,,,,,,,,,,,,,,,,,,
CCMSLIB00000078626,Cc1ccc(C(=O)c2cc([N+](=O)[O-])c(O)c(O)c2)cc1,Tolcapone,1,"['Tolcapone', '134308-13-7', 'Tasmar', 'Ro 40-...",4659569.0,CC1=CC=C(C=C1)C(=O)C2=CC(=C(C(=C2)O)O)[N+](=O)...,Cc1ccc(C(=O)c2cc([N+](=O)[O-])c(O)c(O)c2)cc1,4659569.0,"(3,4-dihydroxy-5-nitrophenyl)-(4-methylphenyl)...",3.3,...,OFP,UNII:CIF6334OLY,FDA:1998-01-29,FDA,NaN,4.0,4.0,4.0,True,3
CCMSLIB00000085811,CN1CCCN=C1COC(=O)C(O)(c1ccccc1)C1CCCCC1,Oxyphencyclimine hydrochloride,1,"['Oxyphencyclimine hydrochloride', '125-52-0',...",66061.0,CN1CCCN=C1COC(=O)C(C2CCCCC2)(C3=CC=CC=C3)O.Cl,CN1CCCN=C1COC(=O)C(O)(c1ccccc1)C1CCCCC1,66061.0,"(1-methyl-5,6-dihydro-4H-pyrimidin-2-yl)methyl...",NaN,...,OFM,DRUGBANK_ID:DB00383,FDA,FDA,NaN,4.0,4.0,4.0,True,19
CCMSLIB00000085349,Cc1cc(NC(=O)C2=C(O)c3ccccc3S(=O)(=O)N2C)no1,isoxicam,1,"['isoxicam', '34552-84-6', 'Maxicam', 'Isoxica...",54677972.0,CC1=CC(=NO1)NC(=O)C2=C(C3=CC=CC=C3S(=O)(=O)N2C)O,Cc1cc(NC(=O)C2=C(O)c3ccccc3S(=O)(=O)N2C)no1,54677972.0,"4-hydroxy-2-methyl-N-(5-methyl-1,2-oxazol-3-yl...",3.0,...,NaN,UNII:8XU734C4NG,YEAR INTRODUCED:1983-01-01,YEAR INTRODUCED,NaN,4.0,4.0,4.0,True,19
CCMSLIB00005723367,CC(C)[C@H](N)C(=O)O,L-valine,2,"['L-valine', 'valine', '72-18-4', '(S)-Valine'...",6287.0,CC(C)[C@@H](C(=O)O)N,CC(C)C(N)C(=O)O,6287.0,(2S)-2-amino-3-methylbutanoic acid,-2.3,...,NaN,UNII:HG18B9YRS7,FDA,FDA,NaN,4.0,4.0,4.0,True,19
CCMSLIB00005788030,CC(C)[C@H](N)C(=O)O,L-valine,2,"['L-valine', 'valine', '72-18-4', '(S)-Valine'...",6287.0,CC(C)[C@@H](C(=O)O)N,CC(C)C(N)C(=O)O,6287.0,(2S)-2-amino-3-methylbutanoic acid,-2.3,...,NaN,UNII:HG18B9YRS7,FDA,FDA,NaN,4.0,4.0,4.0,True,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CCMSLIB00005768836,O=c1ccc2cc3c(cc2o1)occ3,Psoralen,1,"['Psoralen', '66-97-7', 'Ficusin', '7H-Furo[3,...",6199.0,C1=CC(=O)OC2=CC3=C(C=CO3)C=C21,O=c1ccc2cc3c(cc2o1)occ3,6199.0,"furo[3,2-g]chromen-7-one",2.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,True,45
CCMSLIB00005768316,O=c1ccc2cc3c(cc2o1)occ3,Psoralen,1,"['Psoralen', '66-97-7', 'Ficusin', '7H-Furo[3,...",6199.0,C1=CC(=O)OC2=CC3=C(C=CO3)C=C21,O=c1ccc2cc3c(cc2o1)occ3,6199.0,"furo[3,2-g]chromen-7-one",2.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,True,45
CCMSLIB00005767992,O=c1ccc2cc3c(cc2o1)occ3,Psoralen,1,"['Psoralen', '66-97-7', 'Ficusin', '7H-Furo[3,...",6199.0,C1=CC(=O)OC2=CC3=C(C=CO3)C=C21,O=c1ccc2cc3c(cc2o1)occ3,6199.0,"furo[3,2-g]chromen-7-one",2.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,True,45


In [17]:
filtered_df = pd.read_csv(df_file, sep="\t")
filtered_df[filtered_df["clinical_phase"]==4].groupby("inchi_key").count()
# filtered_df[filtered_df["drugcentral_administration_number"]==4]

,Cat. No.,Product Name,synonyms,CAS No.,Smiles,pubchem_cid,isomeric_smiles,canonical_smiles,mixed_location_plate1,lib_plate_well,...,drugcentral_synonyms,drugcentral_pharma_type,drugcentral_pharma_class,drugcentral_atc,drugcentral_dosage,drugbank_approved_number,drugcentral_administration_number,any_phase,none,lib_plate_well_unique
inchi_key,,,,,,,,,,,,,,,,,,,,,
AAFJXZWCNVJTMK-GUCUJZIJSA-N,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,1,1
AAKJLRGGTJKAMG-UHFFFAOYSA-N,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
ABJKWBDEJIDSJZ-UHFFFAOYSA-N,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
ACCMWZWAEFYUGZ-UHFFFAOYSA-N,1,1,1,1,1,1,1,1,1,1,...,1,0,0,1,0,1,1,1,1,1
ACEWLPOYLGNNHV-UHFFFAOYSA-N,1,1,1,1,1,1,1,1,1,1,...,1,1,1,0,0,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYGHJZDHTFUPRJ-UHFFFAOYSA-N,1,1,1,1,1,1,1,1,1,1,...,1,1,1,0,1,0,0,1,1,1
ZZHLYYDVIOPZBE-UHFFFAOYSA-N,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0,1,1,1,1,1
ZZORFUFYDOWNEF-UHFFFAOYSA-N,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,0,2,0,2,2,2


In [30]:
filtered_df[filtered_df["clinical_phase"]>0]

,Smiles,compound_name,entries,synonyms,pubchem_cid,isomeric_smiles,canonical_smiles,pubchem_cid_parent,iupac,pubchem_logp,...,drugcentral_status,drugcentral_database_id,drugcentral_date_of_approval,drugcentral_administration,drugcentral_orphan,drugbank_approved_number,drugcentral_administration_number,clinical_phase,any_phase,none
gnps_libid,,,,,,,,,,,,,,,,,,,,,
CCMSLIB00000078626,Cc1ccc(C(=O)c2cc([N+](=O)[O-])c(O)c(O)c2)cc1,Tolcapone,1,"['Tolcapone', '134308-13-7', 'Tasmar', 'Ro 40-...",4659569.0,CC1=CC=C(C=C1)C(=O)C2=CC(=C(C(=C2)O)O)[N+](=O)...,Cc1ccc(C(=O)c2cc([N+](=O)[O-])c(O)c(O)c2)cc1,4659569.0,"(3,4-dihydroxy-5-nitrophenyl)-(4-methylphenyl)...",3.3,...,OFP,UNII:CIF6334OLY,FDA:1998-01-29,FDA,NaN,4.0,4.0,4.0,True,3
CCMSLIB00000085811,CN1CCCN=C1COC(=O)C(O)(c1ccccc1)C1CCCCC1,Oxyphencyclimine hydrochloride,1,"['Oxyphencyclimine hydrochloride', '125-52-0',...",66061.0,CN1CCCN=C1COC(=O)C(C2CCCCC2)(C3=CC=CC=C3)O.Cl,CN1CCCN=C1COC(=O)C(O)(c1ccccc1)C1CCCCC1,66061.0,"(1-methyl-5,6-dihydro-4H-pyrimidin-2-yl)methyl...",NaN,...,OFM,DRUGBANK_ID:DB00383,FDA,FDA,NaN,4.0,4.0,4.0,True,19
CCMSLIB00000085349,Cc1cc(NC(=O)C2=C(O)c3ccccc3S(=O)(=O)N2C)no1,isoxicam,1,"['isoxicam', '34552-84-6', 'Maxicam', 'Isoxica...",54677972.0,CC1=CC(=NO1)NC(=O)C2=C(C3=CC=CC=C3S(=O)(=O)N2C)O,Cc1cc(NC(=O)C2=C(O)c3ccccc3S(=O)(=O)N2C)no1,54677972.0,"4-hydroxy-2-methyl-N-(5-methyl-1,2-oxazol-3-yl...",3.0,...,NaN,UNII:8XU734C4NG,YEAR INTRODUCED:1983-01-01,YEAR INTRODUCED,NaN,4.0,4.0,4.0,True,19
CCMSLIB00005723367,CC(C)[C@H](N)C(=O)O,L-valine,2,"['L-valine', 'valine', '72-18-4', '(S)-Valine'...",6287.0,CC(C)[C@@H](C(=O)O)N,CC(C)C(N)C(=O)O,6287.0,(2S)-2-amino-3-methylbutanoic acid,-2.3,...,NaN,UNII:HG18B9YRS7,FDA,FDA,NaN,4.0,4.0,4.0,True,19
CCMSLIB00005788030,CC(C)[C@H](N)C(=O)O,L-valine,2,"['L-valine', 'valine', '72-18-4', '(S)-Valine'...",6287.0,CC(C)[C@@H](C(=O)O)N,CC(C)C(N)C(=O)O,6287.0,(2S)-2-amino-3-methylbutanoic acid,-2.3,...,NaN,UNII:HG18B9YRS7,FDA,FDA,NaN,4.0,4.0,4.0,True,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CCMSLIB00005768836,O=c1ccc2cc3c(cc2o1)occ3,Psoralen,1,"['Psoralen', '66-97-7', 'Ficusin', '7H-Furo[3,...",6199.0,C1=CC(=O)OC2=CC3=C(C=CO3)C=C21,O=c1ccc2cc3c(cc2o1)occ3,6199.0,"furo[3,2-g]chromen-7-one",2.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,True,45
CCMSLIB00005768316,O=c1ccc2cc3c(cc2o1)occ3,Psoralen,1,"['Psoralen', '66-97-7', 'Ficusin', '7H-Furo[3,...",6199.0,C1=CC(=O)OC2=CC3=C(C=CO3)C=C21,O=c1ccc2cc3c(cc2o1)occ3,6199.0,"furo[3,2-g]chromen-7-one",2.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,True,45
CCMSLIB00005767992,O=c1ccc2cc3c(cc2o1)occ3,Psoralen,1,"['Psoralen', '66-97-7', 'Ficusin', '7H-Furo[3,...",6199.0,C1=CC(=O)OC2=CC3=C(C=CO3)C=C21,O=c1ccc2cc3c(cc2o1)occ3,6199.0,"furo[3,2-g]chromen-7-one",2.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,True,45


In [31]:
df_file_mce = r"C:\git\msn_library\data\final_tables\mce_library_all_phase.tsv"
df = pd.read_csv(df_file_mce, sep="\t")

In [44]:
s = {}
for inchikey, clinical in zip(df["inchi_key"], df["clinical_phase"]):
    s[inchikey] = clinical
for inchikey, clinical in zip(filtered_df["inchi_key"], filtered_df["clinical_phase"]):
    s[inchikey] = clinical
s

{'MIQPIUSUKVNLNT-UHFFFAOYSA-N': 4.0,
 'ZKFQEACEUNWPMT-UHFFFAOYSA-N': 3.0,
 'OIRFJRBSRORBCM-UHFFFAOYSA-N': 4.0,
 'KLZWOWYOHUKJIG-BPUTZDHNSA-N': 4.0,
 'QKDRXGFQVGOQKS-CRSSMBPESA-N': 4.0,
 'SOYCMDCMZDHQFP-UHFFFAOYSA-N': 4.0,
 'SGTNSNPWRIOYBX-HHHXNRCGSA-N': 4.0,
 'ZCCUWMICIWSJIX-NQJJCJBVSA-O': 4.0,
 'IKGXIBQEEMLURG-NVPNHPEKSA-N': 4.0,
 'YTZALCGQUPRCGW-ZSFNYQMMSA-N': 4.0,
 'PBCZLFBEBARBBI-UHFFFAOYSA-N': 4.0,
 'PYMYPHUHKUWMLA-WDCZJNDASA-N': 4.0,
 'KWIUHFFTVRNATP-UHFFFAOYSA-O': 4.0,
 'HMEYVGGHISAPJR-IAHYZSEUSA-N': 4.0,
 'ROHFNLRQFUQHCH-YFKPBYRVSA-N': 4.0,
 'IQFYYKKMVGJFEH-XLPZGREQSA-N': 4.0,
 'YPELFRMCRYSPKZ-UHFFFAOYSA-N': 4.0,
 'METKIMKYRPQLGS-GFCCVEGCSA-N': 4.0,
 'ZDXPYRJPNDTMRX-GSVOUGTGSA-N': 4.0,
 'HPHUVLMMVZITSG-UHFFFAOYSA-N': 4.0,
 'UYIFTLBWAOGQBI-BZDYCCQFSA-N': 4.0,
 'NENPYTRHICXVCS-YNEHKIRRSA-N': 4.0,
 'ODUOJXZPIYUATO-UHFFFAOYSA-N': 4.0,
 'RXPRRQLKFXBCSJ-GIVPXCGWSA-N': 4.0,
 'RHGKLRLOHDJJDR-BYPYZUCNSA-N': 4.0,
 'AHFWIQIYAXSLBA-RQXATKFSSA-N': 4.0,
 'NCLGDOBQAWBXRA-PGRDOPGGSA-N': 4.0,
 

In [46]:
sum([value==4 for key, value in s.items()])

3420

In [47]:
3420/2275

1.5032967032967033

In [40]:
filtered_df.loc["CCMSLIB00006119686", "canonical_smiles"]

'NC(=O)c1nn(C2OC(CO)C(O)C2O)cn1'

In [5]:
spectra = []
columns = ["compound_name", "canonical_smiles", "isomeric_smiles", "inchi", "inchi_key", "synonyms", "pubchem_cid", "iupac", "formula",
           "unii", "chembl_id", "drugbank_id", "drugcentral_id", "exact_mass", "clinical_phase",
           "drugcentral_administration", "drugbank_approved"]

with pyteomics.mgf.MGF(infile) as f_in:
    for spectrum_dict in tqdm(f_in):
        libid = spectrum_dict["params"]["spectrumid"]
        if libid in filtered_df.index:
            for col in columns:
                value = filtered_df.loc[libid, col]
                spectrum_dict["params"][col] = value if pd.notnull(value) else ""

            spectra.append(spectrum_dict)

pyteomics.mgf.write(spectra, r'C:\git\msn_library\data\final_tables\gnps_drugs_clinical_phase_greater0.mgf')

0it [00:00, ?it/s]